# Brazil Capacity Problem
Make sure you've installed PuLP, Pandas and Plotly.  If you haven't done this, go down two blocks for directions
Instructions for running are in the block directly below this one.

In [1]:
# This is the block you should make changes
# Only change things to the right of the '=' sign. 
# Note that the '#' symbol means a comment-- when the code runs, it ignores this.
# Here you can change the scnario name, number of warehouses, the distance bands, and whether you draw the maps

# To Run, go to the menu item Cell and select 'Run All'
# To see the results of the run: 
#      Go to the directory where you put this file and you'll see two output files
#      One output file will be the summary and one will have details
#      And, tabs will open if you decide to map the input and output

# You should change the name to the right of the '=' sign. This names each scenario. 
# If you don't change it, you'll overwrite the files when you run.  Don't put spaces in the name
# And, kep the single quotes in the name.  

scenario_name = 'scenario_1'  #<<<----- you can change this name

# Change the capacity of a warehouse you'd like to select. This capacity would be same across all warehouses.

warehouse_capacity = 30000000

# Change the number of warehouses you would like to select.  This number should be an integer

number_of_whs = 5  #<<<------ you can change this number

# If you don't want either the input or output map to draw, type False with a capital F instead of True

input_map = True   #<<<----- you can change these from True to False
output_map = True

# To Run, go to the menu item Cell and select 'Run All'


In [2]:
# We need to add directions for installing PuLP, Pandas, and Plotly here. We'll also include these instructions with the overall install 

In [3]:
#This is code to start running the model.  You don't need to interact with this

from pulp import *
import time
import pandas as pd
import plotly
plotly.offline.init_notebook_mode()
import plotly.graph_objs as go

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [4]:
# This model minimizes the total distance

def optimal_location(warehouses, customers, customer_demands, distance, scenario_name):
    start_time=time.time()
    # Create the 'prob' variable to contain the problem data
    brazil_problem = LpProblem("brazil", LpMinimize)

    # A dictionary called 'city_vars' is created to contain the decision variables Yi,j - the assignment of the city to a facility
    # lower bound is 0; upper bound is 1; integer variable - this makes it binary
    assign_vars = LpVariable.dicts("Asssignment",[(w, c) for w in warehouses for c in customers],0,1,LpInteger)
    # Another dicionary called 'facility_vars' is created to contain the decision variables Xi - the decision of a facility at city i to be opened or not
    facility_vars = LpVariable.dicts("Open", [w for w in warehouses],0,1,LpInteger)

    # The objective function is added to 'prob' first
    total_distance = lpSum([distance[w,c]*customer_demands[c]*assign_vars[w,c] 
                       for w in warehouses for c in customers])

    # The six constraints are added to 'brazil_problem'

    """
    
    A Pulp LP constraint: LpConstraint(e, sense, name, rhs)
    Parameters:	
    e – an instance of LpExpression
    sense – one of LpConstraintEQ, LpConstraintGE, LpConstraintLE
    name – identifying string
    rhs – numerical value of constraint right hand side
    """
    
    # Every customer must be served
    for customer in customers:
        brazil_problem += LpConstraint(e = lpSum([assign_vars[w, customer] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name=str(customer)+"_Served", 
                                    rhs=1)
        
    # Ensure that P and exactly P facilities are opened.  We use the variable name 'number_of_whs' for P
    brazil_problem += LpConstraint(e = lpSum([facility_vars[w] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name="FacilityCount", 
                                    rhs=number_of_whs)
    
    # Ensures that the demand served by a warehouse doesn't exceed its capacity
    for facility in warehouses:
        brazil_problem += LpConstraint(e = lpSum([customer_demands[c]*assign_vars[facility, c] for c in customers]) - warehouse_capacity*facility_vars[facility] , 
                                sense=LpConstraintLE, 
                                name=str(facility) + "_Capacity",
                                rhs=0)

    # An assignment can only exist between facility and customer if the facility is opened. We'll call this the "route"
    for facility in warehouses:
        for customer in customers:
            brazil_problem += LpConstraint(e = assign_vars[facility, customer]-facility_vars[facility] , 
                                    sense=LpConstraintLE, 
                                    name=str(facility) + "_" + str(customer) + "_Route",
                                    rhs=0)

    # Setting problem objective
    brazil_problem.setObjective(total_distance)
    # The problem data is written to an .lp file
    brazil_problem.writeLP("brazil_prob.lp")

    # The problem is solved using PuLP's choice of Solver
    brazil_problem.solve()
    
    # The status of the solution is printed to the screen
    #print ("Status:", LpStatus[brazil_problem.status])
    file.write('\nstatus:'+ LpStatus[brazil_problem.status])
    print("Optimization Status",LpStatus[brazil_problem.status] )  #print in Jupyter Notebook
    #print objective
    total_demand = sum(customer_demands.values())
    total_demand_to_warehouse = {w: sum(customer_demands[c]*assign_vars[w,c].varValue for c in customers)
                                    for w in warehouses
                                    if facility_vars[w].varValue>0}
    
    #print (total_demand_to_warehouse)
    #print ("Total Demand",total_demand)
    file.write("\nTotal Demand:"+ str(total_demand))
    print("Objective: ", value(brazil_problem.objective))
    file.write("\nObjective: "+ str(value(brazil_problem.objective)))
    print("Average service distance: ",round(value(brazil_problem.objective)/total_demand,1),"km")
    file.write("\nAverage service distance: "+str(round(value(brazil_problem.objective)/total_demand,1))+"km")
    
    end_time = time.time()
    time_diff = end_time - start_time
    file.write("\nRun Time of model in seconds {:.1f}" .format(time_diff))
    print("Run Time of model in seconds {:.1f}" .format(time_diff))
 
    
    #preparing data to write in excel sheets
    opened_warehouses = []
      
    for w in facility_vars.keys():
        if(facility_vars[w].varValue > 0):
            #print(warehouses[w])
            wh = {
                'Warehouse Key': w,
                'Warehouse City':warehouses[w][1],
                'Country':warehouses[w][2],
                'Lat':warehouses[w][3],
                'Lon':warehouses[w][4],
                'Total Demand to Warehouse':total_demand_to_warehouse[w]
            }
            opened_warehouses.append(wh)
            
            
    #converting the list to dataframe        
    df_wh = pd.DataFrame.from_records(opened_warehouses)
    
    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'Country', 'Total Demand to Warehouse']]
    list_warehouses_open = list(df_wh['Warehouse Key'])
    
    # writing detailed files
    writer = pd.ExcelWriter(scenario_name+'_detailed.xlsx')
    df_wh.to_excel(writer,'Opened Warehouses',index=False)
    
#     print("Customers Assigned to warehouses")
    customers_assignment = []   
    for (w,c) in assign_vars.keys():
        if assign_vars[(w,c)].varValue > 0:
            cust = {
                'Warehouse':str(warehouses[w][1]+','+warehouses[w][2]),
                'Customer':str(customers[c][1]+','+customers[c][2]),
                'Customer Demand': customer_demands[c],
                'Distance': distance[w,c],
                'Warehouse Latitude' : warehouses[w][3],
                'Warehouse Longitude' : warehouses[w][4],
                'Customers Latitude' : customers[c][3],
                'Customers Longitude': customers[c][4]
            }
            customers_assignment.append(cust)
                  
    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse', 'Customer', 'Distance', 'Customer Demand']]
    df_cu.to_excel(writer,'Customers Assignment',index=False)
    
    writer.close()   
    return df_cu_copy,list_warehouses_open


In [5]:
#  This is the input data.  In a normal project, you would load this from another file.  We are making it simple by packaging it here 
def get_data():
    # Warehouses:: 
    # id:(name, city, country, lat, long)
    warehouses = {5: ('Anápolis', 'Anápolis', 'Brazil', -16.32, -48.96),
    19: ('Belém', 'Belém', 'Brazil', -1.44, -48.5),
    22: ('Betim', 'Betim', 'Brazil', -19.97, -44.19),
    32: ('Campina Grande', 'Campina Grande', 'Brazil', -7.23, -35.88),
    34: ('Campo Grande', 'Campo Grande', 'Brazil', -20.45, -54.63),
    38: ('Cariacica', 'Cariacica', 'Brazil', -20.23, -40.37),
    40: ('Cascavel', 'Cascavel', 'Brazil', -24.96, -53.46),
    43: ('Caxias do Sul', 'Caxias do Sul', 'Brazil', -29.18, -51.17),
    49: ('Cuiabá', 'Cuiabá', 'Brazil', -15.61, -56.09),
    94: ('Juiz de Fora', 'Juiz de Fora', 'Brazil', -21.75, -43.36),
    103: ('Maceió', 'Maceió', 'Brazil', -9.65, -35.75),
    105: ('Manaus', 'Manaus', 'Brazil', -3.12, -60.02),
    113: ('Montes Claros', 'Montes Claros', 'Brazil', -16.72, -43.86),
    115: ('Natal', 'Natal', 'Brazil', -5.8, -35.22),
    125: ('Palmas', 'Palmas', 'Brazil', -10.27, -48.31),
    137: ('Piracicaba', 'Piracicaba', 'Brazil', -22.71, -47.64),
    139: ('Ponta Grossa', 'Ponta Grossa', 'Brazil', -25.09, -50.16),
    142: ('Porto Velho', 'Porto Velho', 'Brazil', -8.76, -63.91),
    147: ('Recife', 'Recife', 'Brazil', -8.08, -34.92),
    157: ('Salvador', 'Salvador', 'Brazil', -12.97, -38.5),
    160: ('Santa Maria', 'Santa Maria', 'Brazil', -29.69, -53.83),
    164: ('Santos', 'Santos', 'Brazil', -23.95, -46.33),
    170: ('São José do Rio Preto', 'São José do Rio Preto', 'Brazil', -20.8, -49.39),
    189: ('Teresina', 'Teresina', 'Brazil', -5.1, -42.8),
    193: ('Uberlândia', 'Uberlândia', 'Brazil', -18.9, -48.28)}

    # customers:: 
    # id:(name, city, country, lat, long)
    customers = {1: ('São Paulo Region', 'São Paulo Region', 'Brazil', -23.53, -46.63),
    2: ('Rio de Janeiro Region', 'Rio de Janeiro Region', 'Brazil', -22.91, -43.2),
    3: ('Minas Gerais Region', 'Minas Gerais Region', 'Brazil', -19.92, -43.94),
    4: ('Bahia Region', 'Bahia Region', 'Brazil', -12.97, -38.5),
    5: ('Paraná Region', 'Paraná Region', 'Brazil', -25.42, -49.29),
    6: ('Rio Grande do Sul Region', 'Rio Grande do Sul Region', 'Brazil', -30.04, -51.22),
    7: ('Pernambuco Region', 'Pernambuco Region', 'Brazil', -8.08, -34.92),
    8: ('Ceará Region', 'Ceará Region', 'Brazil', -3.78, -38.59),
    9: ('Santa Catarina Region', 'Santa Catarina Region', 'Brazil', -26.32, -48.84),
    10: ('Pará Region', 'Pará Region', 'Brazil', -1.44, -48.5),
    11: ('Goiás Region', 'Goiás Region', 'Brazil', -16.72, -49.26),
    12: ('Distrito Federal Region', 'Distrito Federal Region', 'Brazil', -15.78, -47.91),
    13: ('Espírito Santo Region', 'Espírito Santo Region', 'Brazil', -20.13, -40.32),
    14: ('Amazonas Region', 'Amazonas Region', 'Brazil', -3.12, -60.02),
    15: ('Maranhão Region', 'Maranhão Region', 'Brazil', -2.5, -44.3),
    16: ('Alagoas Region', 'Alagoas Region', 'Brazil', -9.65, -35.75),
    17: ('Rio Grande do Norte Region', 'Rio Grande do Norte Region', 'Brazil', -5.8, -35.22),
    18: ('Paraíba Region', 'Paraíba Region', 'Brazil', -7.12, -34.86),
    19: ('Mato Grosso Region', 'Mato Grosso Region', 'Brazil', -15.61, -56.09),
    20: ('Mato Grosso do Sul Region', 'Mato Grosso do Sul Region', 'Brazil', -20.45, -54.63),
    21: ('Piauí Region', 'Piauí Region', 'Brazil', -5.1, -42.8),
    22: ('Sergipe Region', 'Sergipe Region', 'Brazil', -10.91, -37.07),
    23: ('Amapá Region', 'Amapá Region', 'Brazil', 0.04, -51.05),
    24: ('Rondônia Region', 'Rondônia Region', 'Brazil', -8.76, -63.91),
    25: ('Acre Region', 'Acre Region', 'Brazil', -9.98, -67.82)}

    
    #  demand::
    # id: demand_value
    customer_demands = {1: 29029226, 2: 13370786, 3: 8663076, 4: 5899854, 5: 4947330, 6: 4912164, 7: 3921090, 8: 3579449, 9: 2657192, 10: 2597519, 11: 2559855, 12: 2551909,
                        13: 1896478, 14: 1865277, 15: 1383837, 16: 1330514, 17: 1239777, 18: 1230034, 19: 1184391, 20: 1010989, 21: 980832, 22: 735616, 23: 440064, 24: 340522, 25: 338812}
        
    # (from, to): distance
    distance = {(5, 1): 837.0, (5, 2): 948.0, (5, 3): 664.0, (5, 4): 1184.0, (5, 5): 1012.0, (5, 6): 1542.0, (5, 7): 1777.0, (5, 8): 1795.0, (5, 9): 1111.0, (5, 10): 1654.0, (5, 11): 55.0, (5, 12): 127.0, (5, 13): 1005.0, (5, 14): 1900.0, (5, 15): 1618.0, (5, 16): 1610.0, (5, 17): 1898.0, (5, 18): 1842.0, (5, 19): 766.0, (5, 20): 754.0, (5, 21): 1416.0, (5, 22): 1417.0, (5, 23): 1832.0, (5, 24): 1825.0, (5, 25): 2157.0,
                (19, 1): 2463.0, (19, 2): 2453.0, (19, 3): 2112.0, (19, 4): 1689.0, (19, 5): 2666.0, (19, 6): 3191.0, (19, 7): 1674.0, (19, 8): 1130.0, (19, 9): 2765.0, (19, 10): 0.0, (19, 11): 1700.0, (19, 12): 1595.0, (19, 13): 2259.0, (19, 14): 1293.0, (19, 15): 481.0, (19, 16): 1678.0, (19, 17): 1550.0, (19, 18): 1637.0, (19, 19): 1781.0, (19, 20): 2215.0, (19, 21): 752.0, (19, 22): 1642.0, (19, 23): 328.0, (19, 24): 1888.0, (19, 25): 2335.0,
                (22, 1): 469.0, (22, 2): 342.0, (22, 3): 27.0, (22, 4): 986.0, (22, 5): 800.0, (22, 6): 1323.0, (22, 7): 1655.0, (22, 8): 1898.0, (22, 9): 850.0, (22, 10): 2112.0, (22, 11): 645.0, (22, 12): 609.0, (22, 13): 404.0, (22, 14): 2540.0, (22, 15): 1941.0, (22, 16): 1461.0, (22, 17): 1849.0, (22, 18): 1746.0, (22, 19): 1348.0, (22, 20): 1090.0, (22, 21): 1659.0, (22, 22): 1262.0, (22, 23): 2345.0, (22, 24): 2457.0, (22, 25): 2764.0,
                (32, 1): 2144.0, (32, 2): 1910.0, (32, 3): 1656.0, (32, 4): 699.0, (32, 5): 2471.0, (32, 6): 2997.0, (32, 7): 142.0, (32, 8): 487.0, (32, 9): 2525.0, (32, 10): 1539.0, (32, 11): 1795.0, (32, 12): 1617.0, (32, 13): 1511.0, (32, 14): 2709.0, (32, 15): 1070.0, (32, 16): 269.0, (32, 17): 175.0, (32, 18): 113.0, (32, 19): 2388.0, (32, 20): 2495.0, (32, 21): 800.0, (32, 22): 429.0, (32, 23): 1865.0, (32, 24): 3088.0, (32, 25): 3521.0,
                (34, 1): 892.0, (34, 2): 1211.0, (34, 3): 1116.0, (34, 4): 1905.0, (34, 5): 777.0, (34, 6): 1119.0, (34, 7): 2524.0, (34, 8): 2538.0, (34, 9): 880.0, (34, 10): 2215.0, (34, 11): 701.0, (34, 12): 879.0, (34, 13): 1491.0, (34, 14): 2012.0, (34, 15): 2287.0, (34, 16): 2351.0, (34, 17): 2652.0, (34, 18): 2592.0, (34, 19): 559.0, (34, 20): 0.0, (34, 21): 2131.0, (34, 22): 2154.0, (34, 23): 2310.0, (34, 24): 1637.0, (34, 25): 1829.0,
                (38, 1): 742.0, (38, 2): 417.0, (38, 3): 374.0, (38, 4): 831.0, (38, 5): 1080.0, (38, 6): 1541.0, (38, 7): 1472.0, (38, 8): 1838.0, (38, 9): 1097.0, (38, 10): 2267.0, (38, 11): 1015.0, (38, 12): 937.0, (38, 13): 12.0, (38, 14): 2854.0, (38, 15): 2016.0, (38, 16): 1276.0, (38, 17): 1697.0, (38, 18): 1573.0, (38, 19): 1738.0, (38, 20): 1485.0, (38, 21): 1702.0, (38, 22): 1094.0, (38, 23): 2534.0, (38, 24): 2829.0, (38, 25): 3151.0,
                (40, 1): 710.0, (40, 2): 1066.0, (40, 3): 1126.0, (40, 4): 2057.0, (40, 5): 422.0, (40, 6): 606.0, (40, 7): 2717.0, (40, 8): 2840.0, (40, 9): 487.0, (40, 10): 2667.0, (40, 11): 1014.0, (40, 12): 1172.0, (40, 13): 1450.0, (40, 14): 2526.0, (40, 15): 2682.0, (40, 16): 2529.0, (40, 17): 2882.0, (40, 18): 2799.0, (40, 19): 1074.0, (40, 20): 515.0, (40, 21): 2482.0, (40, 22): 2328.0, (40, 23): 2790.0, (40, 24): 2113.0, (40, 25): 2251.0,
                (43, 1): 773.0, (43, 2): 1057.0, (43, 3): 1261.0, (43, 4): 2225.0, (43, 5): 457.0, (43, 6): 96.0, (43, 7): 2894.0, (43, 8): 3118.0, (43, 9): 392.0, (43, 10): 3095.0, (43, 11): 1398.0, (43, 12): 1526.0, (43, 13): 1486.0, (43, 14): 3043.0, (43, 15): 3052.0, (43, 16): 2699.0, (43, 17): 3091.0, (43, 18): 2987.0, (43, 19): 1590.0, (43, 20): 1031.0, (43, 21): 2817.0, (43, 22): 2502.0, (43, 23): 3247.0, (43, 24): 2629.0, (43, 25): 2747.0,
                (49, 1): 1324.0, (49, 2): 1575.0, (49, 3): 1371.0, (49, 4): 1916.0, (49, 5): 1299.0, (49, 6): 1679.0, (49, 7): 2447.0, (49, 8): 2321.0, (49, 9): 1407.0, (49, 10): 1781.0, (49, 11): 739.0, (49, 12): 875.0, (49, 13): 1741.0, (49, 14): 1453.0, (49, 15): 1946.0, (49, 16): 2301.0, (49, 17): 2523.0, (49, 18): 2495.0, (49, 19): 0.0, (49, 20): 559.0, (49, 21): 1862.0, (49, 22): 2121.0, (49, 23): 1825.0, (49, 24): 1140.0, (49, 25): 1416.0,
                (94, 1): 389.0, (94, 2): 130.0, (94, 3): 212.0, (94, 4): 1103.0, (94, 5): 728.0, (94, 6): 1210.0, (94, 7): 1767.0, (94, 8): 2062.0, (94, 9): 753.0, (94, 10): 2324.0, (94, 11): 834.0, (94, 12): 818.0, (94, 13): 363.0, (94, 14): 2742.0, (94, 15): 2141.0, (94, 16): 1571.0, (94, 17): 1977.0, (94, 18): 1864.0, (94, 19): 1503.0, (94, 20): 1177.0, (94, 21): 1851.0, (94, 22): 1378.0, (94, 23): 2561.0, (94, 24): 2628.0, (94, 25): 2917.0,
                (103, 1): 1927.0, (103, 2): 1673.0, (103, 3): 1440.0, (103, 4): 475.0, (103, 5): 2261.0, (103, 6): 2776.0, (103, 7): 197.0, (103, 8): 724.0, (103, 9): 2308.0, (103, 10): 1678.0, (103, 11): 1658.0, (103, 12): 1483.0, (103, 13): 1263.0, (103, 14): 2775.0, (103, 15): 1234.0, (103, 16): 0.0, (103, 17): 432.0, (103, 18): 298.0, (103, 19): 2301.0, (103, 20): 2351.0, (103, 21): 927.0, (103, 22): 201.0, (103, 23): 2006.0, (103, 24): 3089.0, (103, 25): 3510.0,
                (105, 1): 2686.0, (105, 2): 2848.0, (105, 3): 2554.0, (105, 4): 2605.0, (105, 5): 2730.0, (105, 6): 3131.0, (105, 7): 2829.0, (105, 8): 2378.0, (105, 9): 2840.0, (105, 10): 1293.0, (105, 11): 1914.0, (105, 12): 1932.0, (105, 13): 2851.0, (105, 14): 0.0, (105, 15): 1746.0, (105, 16): 2775.0, (105, 17): 2763.0, (105, 18): 2819.0, (105, 19): 1453.0, (105, 20): 2012.0, (105, 21): 1921.0, (105, 22): 2673.0, (105, 23): 1056.0, (105, 24): 760.0, (105, 25): 1150.0,
                (113, 1): 810.0, (113, 2): 691.0, (113, 3): 356.0, (113, 4): 711.0, (113, 5): 1118.0, (113, 6): 1658.0, (113, 7): 1364.0, (113, 8): 1548.0, (113, 9): 1184.0, (113, 10): 1772.0, (113, 11): 575.0, (113, 12): 444.0, (113, 13): 532.0, (113, 14): 2323.0, (113, 15): 1581.0, (113, 16): 1177.0, (113, 17): 1535.0, (113, 18): 1447.0, (113, 19): 1311.0, (113, 20): 1207.0, (113, 21): 1296.0, (113, 22): 976.0, (113, 23): 2022.0, (113, 24): 2344.0, (113, 25): 2694.0, (115, 1): 2317.0, (115, 2): 2085.0, (115, 3): 1830.0, (115, 4): 874.0, (115, 5): 2644.0, (115, 6): 3171.0, (115, 7): 255.0, (115, 8): 435.0, (115, 9): 2699.0, (115, 10): 1550.0, (115, 11): 1951.0, (115, 12): 1773.0, (115, 13): 1685.0, (115, 14): 2763.0, (115, 15): 1071.0, (115, 16): 432.0, (115, 17): 0.0, (115, 18): 152.0, (115, 19): 2523.0, (115, 20): 2652.0, (115, 21): 842.0, (115, 22): 603.0, (115, 23): 1872.0, (115, 24): 3178.0, (115, 25): 3616.0, (125, 1): 1484.0, (125, 2): 1506.0, (125, 3): 1170.0, (125, 4): 1109.0, (125, 5): 1687.0, (125, 6): 2217.0, (125, 7): 1489.0, (125, 8): 1291.0, (125, 9): 1784.0, (125, 10): 981.0, (125, 11): 724.0, (125, 12): 614.0, (125, 13): 1390.0, (125, 14): 1516.0, (125, 15): 970.0, (125, 16): 1376.0, (125, 17): 1523.0, (125, 18): 1518.0, (125, 19): 1030.0, (125, 20): 1318.0, (125, 21): 835.0, (125, 22): 1230.0, (125, 23): 1185.0, (125, 24): 1718.0, (125, 25): 2134.0,
                (137, 1): 138.0, (137, 2): 455.0, (137, 3): 493.0, (137, 4): 1450.0, (137, 5): 345.0, (137, 6): 889.0, (137, 7): 2118.0, (137, 8): 2318.0, (137, 9): 419.0, (137, 10): 2365.0, (137, 11): 687.0, (137, 12): 771.0, (137, 13): 809.0, (137, 14): 2552.0, (137, 15): 2274.0, (137, 16): 1925.0, (137, 17): 2303.0, (137, 18): 2207.0, (137, 19): 1186.0, (137, 20): 765.0, (137, 21): 2025.0, (137, 22): 1725.0, (137, 23): 2555.0, (137, 24): 2326.0, (137, 25): 2570.0, (139, 1): 397.0, (139, 2): 747.0, (139, 3): 859.0, (139, 4): 1818.0, (139, 5): 95.0, (139, 6): 560.0, (139, 7): 2486.0, (139, 8): 2671.0, (139, 9): 190.0, (139, 10): 2634.0, (139, 11): 935.0, (139, 12): 1061.0, (139, 13): 1149.0, (139, 14): 2659.0, (139, 15): 2587.0, (139, 16): 2293.0, (139, 17): 2669.0, (139, 18): 2574.0, (139, 19): 1221.0, (139, 20): 689.0, (139, 21): 2356.0, (139, 22): 2093.0, (139, 23): 2794.0, (139, 24): 2326.0, (139, 25): 2508.0, (142, 1): 2464.0, (142, 2): 2709.0, (142, 3): 2478.0, (142, 4): 2810.0, (142, 5): 2411.0, (142, 6): 2708.0, (142, 7): 3187.0, (142, 8): 2850.0, (142, 9): 2516.0, (142, 10): 1888.0, (142, 11): 1816.0, (142, 12): 1903.0, (142, 13): 2830.0, (142, 14): 760.0, (142, 15): 2276.0, (142, 16): 3089.0, (142, 17): 3178.0, (142, 18): 3201.0, (142, 19): 1140.0, (142, 20): 1637.0, (142, 21): 2363.0, (142, 22): 2947.0, (142, 23): 1727.0, (142, 24): 0.0, (142, 25): 450.0,
                (147, 1): 2122.0, (147, 2): 1870.0, (147, 3): 1635.0, (147, 4): 669.0, (147, 5): 2455.0, (147, 6): 2972.0, (147, 7): 0.0, (147, 8): 627.0, (147, 9): 2503.0, (147, 10): 1674.0, (147, 11): 1827.0, (147, 12): 1650.0, (147, 13): 1459.0, (147, 14): 2829.0, (147, 15): 1209.0, (147, 16): 197.0, (147, 17): 255.0, (147, 18): 107.0, (147, 19): 2447.0, (147, 20): 2524.0, (147, 21): 931.0, (147, 22): 393.0, (147, 23): 2001.0, (147, 24): 3187.0, (147, 25): 3615.0, (157, 1): 1453.0, (157, 2): 1211.0, (157, 3): 965.0, (157, 4): 0.0, (157, 5): 1786.0, (157, 6): 2304.0, (157, 7): 669.0, (157, 8): 1021.0, (157, 9): 1834.0, (157, 10): 1689.0, (157, 11): 1228.0, (157, 12): 1060.0, (157, 13): 819.0, (157, 14): 2605.0, (157, 15): 1327.0, (157, 16): 475.0, (157, 17): 874.0, (157, 18): 762.0, (157, 19): 1916.0, (157, 20): 1905.0, (157, 21): 993.0, (157, 22): 277.0, (157, 23): 2000.0, (157, 24): 2810.0, (157, 25): 3208.0, (160, 1): 990.0, (160, 2): 1298.0, (160, 3): 1473.0, (160, 4): 2437.0, (160, 5): 652.0, (160, 6): 254.0, (160, 7): 3107.0, (160, 8): 3295.0, (160, 9): 616.0, (160, 10): 3189.0, (160, 11): 1514.0, (160, 12): 1660.0, (160, 13): 1724.0, (160, 14): 3023.0, (160, 15): 3184.0, (160, 16): 2913.0, (160, 17): 3294.0, (160, 18): 3196.0, (160, 19): 1581.0, (160, 20): 1030.0, (160, 21): 2967.0, (160, 22): 2714.0, (160, 23): 3317.0, (160, 24): 2551.0, (160, 25): 2628.0,
                (164, 1): 56.0, (164, 2): 339.0, (164, 3): 511.0, (164, 4): 1472.0, (164, 5): 341.0, (164, 6): 832.0, (164, 7): 2141.0, (164, 8): 2390.0, (164, 9): 365.0, (164, 10): 2512.0, (164, 11): 859.0, (164, 12): 923.0, (164, 13): 750.0, (164, 14): 2741.0, (164, 15): 2393.0, (164, 16): 1945.0, (164, 17): 2340.0, (164, 18): 2234.0, (164, 19): 1377.0, (164, 20): 938.0, (164, 21): 2128.0, (164, 22): 1749.0, (164, 23): 2714.0, (164, 24): 2517.0, (164, 25): 2755.0, (170, 1): 416.0, (170, 2): 680.0, (170, 3): 576.0, (170, 4): 1447.0, (170, 5): 513.0, (170, 6): 1043.0, (170, 7): 2100.0, (170, 8): 2223.0, (170, 9): 616.0, (170, 10): 2153.0, (170, 11): 454.0, (170, 12): 579.0, (170, 13): 947.0, (170, 14): 2276.0, (170, 15): 2107.0, (170, 16): 1914.0, (170, 17): 2260.0, (170, 18): 2180.0, (170, 19): 912.0, (170, 20): 546.0, (170, 21): 1884.0, (170, 22): 1713.0, (170, 23): 2323.0, (170, 24): 2052.0, (170, 25): 2308.0, (189, 1): 2089.0, (189, 2): 1979.0, (189, 3): 1651.0, (189, 4): 993.0, (189, 5): 2361.0, (189, 6): 2908.0, (189, 7): 931.0, (189, 8): 489.0, (189, 9): 2444.0, (189, 10): 752.0, (189, 11): 1470.0, (189, 12): 1311.0, (189, 13): 1691.0, (189, 14): 1921.0, (189, 15): 333.0, (189, 16): 927.0, (189, 17): 842.0, (189, 18): 905.0, (189, 19): 1862.0, (189, 20): 2131.0, (189, 21): 0.0, (189, 22): 902.0, (189, 23): 1079.0, (189, 24): 2363.0, (189, 25): 2808.0,
                (193, 1): 542.0, (193, 2): 690.0, (193, 3): 469.0, (193, 4): 1235.0, (193, 5): 732.0, (193, 6): 1273.0, (193, 7): 1877.0, (193, 8): 1982.0, (193, 9): 826.0, (193, 10): 1940.0, (193, 11): 263.0, (193, 12): 349.0, (193, 13): 845.0, (193, 14): 2168.0, (193, 15): 1873.0, (193, 16): 1695.0, (193, 17): 2029.0, (193, 18): 1953.0, (193, 19): 906.0, (193, 20): 686.0, (193, 21): 1644.0, (193, 22): 1495.0, (193, 23): 2126.0, (193, 24): 2026.0, (193, 25): 2322.0}
    
    # Accounting for circuity factor
    distance = {(w,c): distance[w,c]*1.17 for w in warehouses for c in customers}
    
    return warehouses, customers, customer_demands, distance


In [6]:
# In this function, we are testing the input paramaters
def test_input(warehouses, customers, customer_demands, distance):
    for c in customers:
        for w in warehouses:
            if(distance[w,c]>=0):
                pass
            else:
                file.write(f'\nDistance between warehouse {w} and customer {c} is not available or invalid')

    for c in customers.keys():
        if(customer_demands[c]>=0):
            pass
        else:
            file.write(f'\nDemand for Customer {c} is not available')
                    

In [7]:
# In this function, we are visualizaing the input data-- the data before the optimization has run
def input_visual(warehouses, customers):
    warehouse_list = []
    for w in warehouses.keys():
        wh = {
                    'text':'Warehouse-'+warehouses[w][1],
                    'Country':warehouses[w][2],
                    'lat':warehouses[w][3],
                    'long':warehouses[w][4],
                    'cnt':10000000,
                    'size' : 30,
                    'color' : 'rgba(0, 100, 0)'
                    }
        warehouse_list.append(wh)    

    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][1],
            'Country':customers[c][2],
            'lat':customers[c][3],
            'long':customers[c][4] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)       

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 

    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'country names',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),

            ))]

    layout = dict(
            title = "Brazil's Capacity - Input",
            showlegend = False, 
            geo = dict(
                scope='south america',
                projection=dict( type='mercator' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    plotly.offline.plot({ "data":locations, "layout":layout}, filename = scenario_name+'_input.html')  

    

In [8]:
# In this function, we are visualizaing the output data-- the data after the optimization has run
def output_visual(warehouses, customers, df_cu, wh_loc, scenario_name):   
    
    warehouse_list = []
    for w in wh_loc:
            wh = {
                        'text':'Warehouse-'+warehouses[w][1],
                        'Country':warehouses[w][2],
                        'lat':warehouses[w][3],
                        'long':warehouses[w][4],
                        'cnt':10000000,
                        'size' : 30,
                        'color' : 'rgba(0, 100, 0)'
                        }
            warehouse_list.append(wh)    
            
    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][1],
            'Country':customers[c][2],
            'lat':customers[c][3],
            'long':customers[c][4] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)
        
    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 

    paths = []        
    for i in range( len( df_cu) ):
            paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'country names',
                    lon = [ df_cu['Warehouse Longitude'][i], df_cu['Customers Longitude'][i] ],
                    lat = [ df_cu['Warehouse Latitude'][i], df_cu['Customers Latitude'][i] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'red',
                    ),
                    opacity = 0.8,
                )
            )  
                

    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'country names',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),
            ))]

    layout = dict(
            title = "Brazil's Capacity - Output",
            showlegend = False, 
            geo = dict(
                scope='south america',
                projection=dict( type='mercator' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )

    plotly.offline.plot({"data":locations+paths, "layout":layout},filename=scenario_name+'_output.html')         

## This is the block of code that runs all the functions (including the optimization)

In [9]:
#getting input data
warehouses, customers, customer_demands, distance = get_data()

#print("loaded input data")
#distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]

#opening text file in write mode
file = open(scenario_name+'summary' + '.txt',"w")
file.write('loaded input data')
print("loaded input data")  #Also print to Jupyter

#testing input data
file.write('\nTesting Input Data')
test_input(warehouses, customers, customer_demands, distance)

#RunCode = ()  #We need some function in here that runs the whole thing.
file.write('\nBuilding the models')

df_assign_vars, list_facility_vars = optimal_location(warehouses, customers, customer_demands, distance, scenario_name)        

#printing the maps
if input_map == True:
    input_visual(warehouses, customers)
if output_map == True:
    output_visual(warehouses, customers, df_assign_vars, list_facility_vars, scenario_name)

#closing the file
file.close()

#Allocation of warehouses to customers
#print (df_assign_vars, "\n\n", list_facility_vars)

loaded input data
Optimization Status Optimal
Objective:  33419025354.779995
Average service distance:  338.7 km
Run Time of model in seconds 0.2
